# Formatting the LPP txt files

In this notebook, we'll go from the raw LPP txt files, to a word-based csv file, doing the following steps:
- Tokenizing the natural language by words
- Remove the blank space between a word and :
- Adding capital letters at the beginning of a sentence
- Remove the blank space between - and the following word (dialogue) 

### Bash commands preprocessing

In [ ]:
# For :
!perl -pi.bak -e 's/ :/:/g' *.txt

# For dash
!perl -pi.bak -e 's/- /-/g' *.txt

# Word tokenizing
!for f in `seq 1 9` ; do sed 's/ /\n/g' text_french_run$f.txt | awk 'length($0) > 0 ' > new_test_run$f.txt; done


### Python commands tokenizing

In [100]:
import pandas as pd
import numpy as np
import copy

In [131]:
for i in np.arange(1,10):
    with open(f'./text_lpp/new_test_run{i}.txt') as temp_file:

        lpp = temp_file.read().splitlines() 


    df = pd.DataFrame(lpp)
    next_cap = False

    for index, word in df.iterrows():
        # First word
        if index == 0:
            df.at[index,0] = str(word.str.capitalize()[0])
        if next_cap == True:
            df.at[index,0] = str(word.str.capitalize()[0])
        if str(row.word).__contains__('.') or str(row.word).__contains__('?') or str(row.word).__contains__('!'):
            next_cap = True
        else:
            next_cap = False

    df.columns = ['word']
    end = (df.shape[0]*0.35)+0.7
    df['onset'] = np.arange(0.7,end,0.35)
    df['duration'] = np.ones(df.shape[0])*0.3
    
    df.to_csv(f'./txt_clean/run{i}_clean.tsv',sep='\t',index=False)
    
    
    # Create a dataframe where the duration of the black screen after the end of the sentence is longer.

    df_sentence_end = pd.DataFrame(columns = df.columns, data = copy.deepcopy(df.values))
    end_of_sentence_delay = 0.2

    for index, row in df.iterrows():
        if str(row.word).__contains__('.') or str(row.word).__contains__('?') or str(row.word).__contains__('!'):
            # df_sentence_end.at[index, 'onset'] = row.onset + end_of_sentence_delay # Add the delay from this line
            # And for every next onset
            for j in np.arange(index+1, df.shape[0]):
                df_sentence_end.at[j, 'onset'] = df_sentence_end.at[j, 'onset'] + end_of_sentence_delay
                ww = df_sentence_end.at[j, 'word']

    
    df_sentence_end.to_csv(f'./txt_clean_end_of_sentence/run{i}_clean_sentence.tsv',sep='\t',index=False)

        
     